In [1]:
from knockai.log_decode import log_decode
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from knockai.log_decode import clean_data



In [2]:
# Load the data into a Pandas DataFrame
df = log_decode('/Users/danchecketts/dev/learning/Knockai/resources/runs/r02/r2-100k-af.txt')
df.head()

,Secs,RPM,MAP,ETemp,IAT,VOLTS,TPS,OXY,WB_Oxy,INJ_PW,...,Accel_,RadFan,Aux1,Aux1,Orun_Vac,Iat_Rtd,Stagin,IDLEST,TPSCls,Markers
0,0.00,900,32,74,18,13.0,20,0.12,15.1,1.6,...,Off,Off,Off,Off,Off,Off,Off,=,Closed,I
1,0.09,900,32,74,16,13.0,20,0.12,14.7,1.6,...,Off,Off,Off,Off,Off,Off,Off,=,Closed,I
2,0.17,800,32,74,16,12.9,20,0.12,13.9,1.6,...,Off,Off,Off,Off,Off,Off,Off,=,Closed,I
3,0.27,800,32,74,16,12.9,20,0.95,13.8,1.6,...,Off,Off,Off,Off,Off,Off,Off,=,Closed,I
4,0.36,800,33,74,16,12.7,20,0.91,14.0,1.7,...,Off,Off,Off,Off,Off,Off,Off,=,Closed,I


In [3]:
df = clean_data(df)

In [4]:
# Calculate the air density based on MAP and Intake Temp
df["air_density"] = df["MAP"] / (df["IAT"] + 273.15)


In [5]:
# Calculate the air flow rate based on RPM and engine displacement
df["air_flow_rate"] = df["RPM"] * 1809 * 0.001 / 2

# Calculate the intake velocity based on air flow rate and air density
df["intake_velocity"] = df["air_flow_rate"] / df["air_density"]

# Calculate the correction factor based on an exponential decay function of intake velocity
df["correction_factor"] = np.exp(-df["intake_velocity"] / 1000)

In [6]:
# Split the data into features (INJ_PW, intake velocity, and correction factor) and target (WB_Oxy)
X = df[["INJ_PW", "intake_velocity", "correction_factor"]].values
y = df["WB_Oxy"].values.reshape(-1, 1)


In [7]:
# Fit the multiple linear regression model to the data
reg = LinearRegression().fit(X, y)

# Get the coefficients of the regression line
coef = reg.coef_[0]
intercept = reg.intercept_[0]

# The regression line can be represented as y = a1 * INJ_PW + a2 * intake_velocity + a3 * correction_factor + b, where a1, a2, a3 = coefficients and b = intercept
print("y = {:.2f} * INJ_PW + {:.2f} * intake_velocity + {:.2f} * correction_factor + {:.2f}".format(coef[0], coef[1], coef[2], intercept))


y = -0.38 * INJ_PW + -0.00 * intake_velocity + 29.41 * correction_factor + 14.26
